<a href="https://colab.research.google.com/github/andrew3690/F-A_root_project/blob/Brazil/C%C3%B3pia_de_Research_Stocks.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Better and consistent library that makes the same function as the code above



In [1]:
! pip install pyspark py4j
! pip install pyodbc
! pip install pandas

Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable


**Importing modules and pandas library that will be used later.**

In [2]:
from pyspark.sql import SparkSession
import pyspark
import pandas as pd
import pyodbc
import findspark

findspark.init()

In [4]:
# instatianting spark instance
spark = SparkSession.builder.master('local[*]').getOrCreate()

spark

23/07/03 14:56:38 WARN Utils: Your hostname, Linux resolves to a loopback address: 127.0.1.1; using 10.0.2.15 instead (on interface enp0s3)
23/07/03 14:56:38 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/07/03 14:56:39 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Gooogle Colab Version

In [192]:
# from google.colab import drive
# drive.mount('/content/gdrive', force_remount=True) 

" \nfrom google.colab import drive\ndrive.mount('/content/gdrive', force_remount=True) \n"

# Next cells acquire parquet data from remote files, and transpose it to spark format

In [5]:
fille =r'RAW_DATA/Parquet/Balance.parquet'

balance = spark\
        .read.format("parquet")\
        .option("inferSchema", "True")\
        .option("header","True")\
        .parquet(fille)

Creating Tables in order to manipulate data in SQL format

In [6]:
# Balance table
balance.createOrReplaceTempView("Balance")

23/07/03 14:58:18 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


# Note 1: 
'DetToEquity' must be in percentage, try to get it done here or otherwise in Power Bi

---

# Note 2: 
Find a way to calculate Beta values from 
historical data get it from other sources, but try to calculate it localy (Talk with Felipe, about the period of time that will be used as 'ytd' or montlhy). 

# Fixed:
Beta values could be acquired trough yahoo finance

Listing brazilian companies that will be researched and getting hystorical financial data
filters that our Analyst indicates: Raking companies by: EBIT/EV > 0, ROE > 0, **Daily equity < 1.000.000.000**, revenueGrowth > 0 AND Will be done apart :(5-year-history) and all margins must be postive

Alternative operations, while dealing with a little amount of data, and it can be joined here

In [7]:
Balance = spark.sql("""
SELECT  Ticker,
        CAST(Data_de_Reporte as string),
        Moeda,
        CAST(Receita_a_Pagar as string),
        CAST(Receitas_a_Receber as string),
        CAST(Bens_de_Capital as string)
FROM Balance
WHERE Ticker NOT LIKE '%34.SA'
AND Moeda = 'BRL'
ORDER BY Ticker DESC;
""")

In [8]:
Balance.show()

+---------+-------------------+-----+---------------+------------------+---------------+
|   Ticker|    Data_de_Reporte|Moeda|Receita_a_Pagar|Receitas_a_Receber|Bens_de_Capital|
+---------+-------------------+-----+---------------+------------------+---------------+
| ZAMP3.SA|2021-12-30 21:00:00|  BRL|      2.47917E8|         1.39267E8|     1.461068E9|
| ZAMP3.SA|2020-12-30 21:00:00|  BRL|      2.17616E8|         1.05011E8|     1.461068E9|
| ZAMP3.SA|2019-12-30 21:00:00|  BRL|      2.86432E8|          6.0361E7|      9.50768E8|
| YDUQ3.SA|2019-12-30 21:00:00|  BRL|      1.26651E8|         7.59622E8|     1.139887E9|
| YDUQ3.SA|2021-12-30 21:00:00|  BRL|      1.83529E8|         9.57746E8|     1.139887E9|
| YDUQ3.SA|2020-12-30 21:00:00|  BRL|      2.51229E8|         8.90151E8|     1.139887E9|
|XPBR31.SA|2019-12-30 21:00:00|  BRL|      2.66813E8|         4.58776E8|        23000.0|
|XPBR31.SA|2022-12-30 21:00:00|  BRL|      6.17394E8|         4.87331E8|        24000.0|
|XPBR31.SA|2021-12-30

ev = número total de papeis x cotação + divida liquida
ev = market value +  liquid debt 

PSR = PREÇO DA AÇÃO / RECEITA LIQUIDA POR AÇÃO 

Pandas setting DataFrame to pandas format:

In [9]:
df_balance = Balance.toPandas()

In [10]:
df_balance

,Ticker,Data_de_Reporte,Moeda,Receita_a_Pagar,Receitas_a_Receber,Bens_de_Capital
0,ZAMP3.SA,2019-12-30 21:00:00,BRL,2.86432E8,6.0361E7,9.50768E8
1,ZAMP3.SA,2020-12-30 21:00:00,BRL,2.17616E8,1.05011E8,1.461068E9
2,ZAMP3.SA,2021-12-30 21:00:00,BRL,2.47917E8,1.39267E8,1.461068E9
3,YDUQ3.SA,2019-12-30 21:00:00,BRL,1.26651E8,7.59622E8,1.139887E9
4,YDUQ3.SA,2020-12-30 21:00:00,BRL,2.51229E8,8.90151E8,1.139887E9
...,...,...,...,...,...,...
749,AERI3.SA,2021-12-30 21:00:00,BRL,4.45286E8,1.26877E8,8.15102E8
750,AERI3.SA,2022-12-30 21:00:00,BRL,3.36048E8,0.0,8.15102E8
751,AALR3.SA,2019-12-30 21:00:00,BRL,5.667E7,8.8789E7,6.12412E8
752,AALR3.SA,2020-12-30 21:00:00,BRL,9.4887E7,7.9679E7,6.12412E8


Excel format tables

In [11]:
# df_stat.to_excel(r"Final/Excel/Stat.xlsx",
#                  sheet_name="stat")

# df_calc.to_excel(r"Final/Excel/Calcl.xlsx",
#                  sheet_name="calc")

# df_rsc.to_excel(r"Final/Excel/RSCH.xlsx",
#                 sheet_name="research")

df_balance.to_excel(r"Final/Excel/Balance.xlsx",
                sheet_name="balance")


Parquet format table

In [163]:
""" 
df_balance.to_parquet(r"C:/Users/video/Desktop/Projetos/API/F-A_root_project/notebooks/Final/Parquet/Balance.parquet")

df_stat.to_parquet(r"C:/Users/video/Desktop/Projetos/API/F-A_root_project/notebooks/Final/Parquet/Calcl.parquet")

df_rsc.to_parquet(r"C:/Users/video/Desktop/Projetos/API/F-A_root_project/notebooks/Final/Parquet/RSCH.parquet")

df_calc.to_parquet(r"C:/Users/video/Desktop/Projetos/API/F-A_root_project/notebooks/Final/Parquet/Calcl.parquet")


df_balance.to_parquet(r"C:/Users/video/Desktop/Projetos/API/F-A_root_project/notebooks/Final/Parquet/Balance.parquet")

df_calendario.to_parquet(r"C:/Users/video/Desktop/Projetos/API/F-A_root_project/notebooks/Final/Parquet/Calendario.parquet")

"""

In [12]:
df_balance['Bens_de_Capital'] = df_balance['Bens_de_Capital'].astype("string")
df_balance['Receitas_a_Receber'] = df_balance['Receitas_a_Receber'].astype("string")
df_balance['Receita_a_Pagar'] = df_balance['Receita_a_Pagar'].astype("string")

print(df_balance.dtypes)

Ticker                        object
Data_de_Reporte               object
Moeda                         object
Receita_a_Pagar       string[python]
Receitas_a_Receber    string[python]
Bens_de_Capital       string[python]
dtype: object
